# Step 1: Install Qdrant and Client


In [5]:
pip install qdrant-client  sentence-transformers


# Ingest Data into Qdrant

In [6]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams
from qdrant_client.models import PointStruct  # Import the correct PointStruct

# Initialize Qdrant client
client = QdrantClient(":memory:")

# Create a new collection in Qdrant
client.recreate_collection(
    collection_name="my_collection",
    vectors_config=VectorParams(size=4, distance="Cosine")  # size is the length of the vector
)

# Sample data: vectors and payloads
vectors = [
    [0.1, 0.2, 0.3, 0.4],
    [0.2, 0.3, 0.4, 0.5],
    [0.9, 0.8, 0.7, 0.6]
]

payloads = [
    {"category": "electronics", "name": "smartphone"},
    {"category": "electronics", "name": "laptop"},
    {"category": "furniture", "name": "chair"}
]

# Ingest vectors into the collection using PointStruct
client.upsert(
    collection_name="my_collection",
    points=[
        PointStruct(id=i, vector=vectors[i], payload=payloads[i])
        for i in range(len(vectors))
    ]
)



<ipython-input-6-2c2f8ed14d37>:9: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# Step 3: Perform a Vector Search

In [7]:
from qdrant_client.http.models import Filter, FieldCondition, MatchValue

# Define the query vector
query_vector = [0.2, 0.3, 0.4, 0.5]

# Define a filter to search only within the "electronics" category
query_filter = Filter(
    must=[
        FieldCondition(
            key="category",
            match=MatchValue(value="electronics")
        )
    ]
)

# Perform the search
search_result = client.search(
    collection_name="my_collection",
    query_vector=query_vector,
    query_filter=query_filter,
    limit=2
)

# Process and print the results
for scored_point in search_result:
    print(f"ID: {scored_point.id}, Score: {scored_point.score}")
    print(f"Payload: {scored_point.payload}")
    print("---")


ID: 1, Score: 1.0000000363917672
Payload: {'category': 'electronics', 'name': 'laptop'}
---
ID: 0, Score: 0.9938079894227599
Payload: {'category': 'electronics', 'name': 'smartphone'}
---
